In [1]:
import numpy as np
import matplotlib.pyplot as plt

# from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%matplotlib widget

## Theoretical Background for the Lattice Visualization Script

The provided script is a visual exploration tool for understanding lattice structures, reciprocal lattice vectors, and the Brillouin Zone in two dimensions. Below is the theoretical backdrop pertaining to the script:

### Lattice Structures in 2D:
A lattice is a periodic arrangement of points in space, with each point having an identical surrounding to every other point. In two dimensions, a lattice can be described by two non-collinear vectors $ \mathbf{a}_1 $ and $ \mathbf{a}_2 $, called the primitive lattice vectors. These vectors define a unit cell which, when repeated periodically in space, forms the entire lattice.

### Reciprocal Lattice:
The reciprocal lattice is a construct that plays a crucial role in the analysis of wave phenomena in crystalline materials. It is defined such that the dot product of a real space lattice vector and a reciprocal space lattice vector equals $2\pi$ times an integer. The reciprocal lattice vectors $ \mathbf{b}_1 $ and $ \mathbf{b}_2 $ in 2D are given by:

$$
\mathbf{b}_1 = 2\pi \frac{\mathbf{R} \cdot \mathbf{a}_2}{\mathbf{a}_1 \cdot (\mathbf{R} \cdot \mathbf{a}_2)}
$$

$$
\mathbf{b}_2 = 2\pi \frac{\mathbf{R} \cdot \mathbf{a}_1}{\mathbf{a}_2 \cdot (\mathbf{R} \cdot \mathbf{a}_1)}
$$

where $ \mathbf{R} $ is a 90-degree rotation matrix.

### Brillouin Zone:
The first Brillouin Zone is defined as the set of points in reciprocal space that are closer to the origin of the reciprocal lattice than to any other reciprocal lattice points. It's a crucial concept for understanding wave-vector dependent phenomena in crystals, like electron and phonon dispersion relations.

### Script Functions:
- `get_reciprocal_vectors(a1, a2)`: Computes the reciprocal lattice vectors based on the input real space lattice vectors.
- `plot_lattice(v1, v2, nb_x, nb_y, add_brillouin)`: Plots the real space lattice and optionally the first Brillouin zone.
- `interactive_lattice_plot(a1, a2, theta)`: Interactive function to visualize the real and reciprocal space lattices based on the user's input for the lattice parameters and the angle between the lattice vectors.

### Script Workflow:
1. The script initializes with an interactive widget allowing the user to specify the norms of the real space lattice vectors $a_1$ and $a_2$, and the angle $ \theta $ between them.
2. Upon interaction, `interactive_lattice_plot` function is called, which computes the real space lattice vectors $ \mathbf{v}_1 $ and $ \mathbf{v}_2 $ based on the provided parameters.
3. `plot_lattice` function is then called twice, once for plotting the real space lattice and once for plotting the reciprocal space lattice.
4. Optionally, the first Brillouin zone can also be plotted in the real space lattice plot.

This script serves as a valuable tool for visualizing and understanding the fundamental concepts of lattice structures, reciprocal lattice, and Brillouin zone in two-dimensional crystalline materials.


In [2]:
def get_reciprocal_vectors(a1,a2):
    '''
    :param a1,a2: Real Space 2D lattice vectors
    :return: b1,b2 : Reciprocal Space basis vector
    '''
    
    # Define the 90 degree rotation matrix
    R =[[0, -1],[1,0]]
    
    # Create to vectors b1,b2 such that ai*bj = 2\pi \delta_{i,j}
    # we first ensure the orthogonality of b1(2) wrt a2,(1) and then normalize the vector
    b1 = 2*np.pi* np.matmul(R,a2)/np.matmul(np.transpose(a1),np.matmul(R,a2))
    b2 = 2*np.pi* np.matmul(R,a1)/np.matmul(np.transpose(a2),np.matmul(R,a1))
    
    return b1,b2



In [3]:
def plot_lattice(v1,v2,nb_x,nb_y,add_brillouin=False):
    '''
    :param v1,v2: lattice vectors
    :parma nb_x,nb_y: (int) Control the number of points by setting the number of times v1 and v2 are repeated
    :param add_brillouin: (bool) if True, show the 1st Brillouin zone
    :return: matplotlib figure
    '''
    coordinates = []
    # Set up the coordinates 
    for ii in np.arange(-nb_x+1,nb_x):
        for jj in np.arange(-nb_y+1,nb_y):
            coordinates.append(ii*v1 + jj*v2)
    
    #Plot the unit cell
    if not(add_brillouin):
        edges = ([(0,0),(v1[0,0],v1[1,0]),(v1[0,0]+v2[0,0],v1[1,0]+v2[1,0]),(v2[0,0],v2[1,0])])
        x,y = zip(*edges)
        plt.fill(x, y, color='#6699cc', alpha=0.7,linewidth=3, zorder=2)
        
    # Plot the first Brillouin zone
    if add_brillouin:
        # Note, we decided to keep v1 horizontal, so the extreme left and right xcoordinates are simply given by : 
        x_left = -v1[0,0]/2
        x_right = v1[0,0]/2
        x_interval = np.linspace(x_left,x_right,100)
        
        # we need to find the equations of the 3 perpendicular bissector 
        #slope is given by -1/m
        m1 = -((v2[1,0])/(v2[0,0]-v1[0,0]))**(-1)
        m2 = -(v2[1,0]/v2[0,0])**(-1)
        m3 = -(v2[1,0]/(v2[0,0]+v1[0,0]))**(-1)
        # Middle of the segment
        M1 = 0.5*(v2-v1)
        M2=0.5*v2
        M3 = 0.5*(v2+v1)
        # Origin
        b1 = M1[1,0]-m1*M1[0,0]
        b2 = M2[1,0]-m2*M2[0,0]
        b3 = M3[1,0]-m3*M3[0,0]
        y_up = lambda x : np.minimum (x*m1+b1,np.minimum(m2*x+b2,x*m3+b3 ))
        
        # Now we can do the same for the bottom (not the best way, but let's keep it simple)
        m1_d = -((-v2[1,0])/(-v2[0,0]-v1[0,0]))**(-1)
        m2_d = -(-v2[1,0]/-v2[0,0])**(-1)
        m3_d = -(-v2[1,0]/(-v2[0,0]+v1[0,0]))**(-1)
        # Middle of the segment
        M1_d = 0.5*(-v2-v1)
        M2_d=0.5*-v2
        M3_d = 0.5*(-v2+v1)
        # Origin
        b1_d = M1_d[1,0]-m1_d*M1_d[0,0]
        b2_d = M2_d[1,0]-m2_d*M2_d[0,0]
        b3_d = M3_d[1,0]-m3_d*M3_d[0,0]
        
        y_down = lambda x : np.maximum (x*m1_d+b1_d,np.maximum(m2_d*x+b2_d,x*m3_d+b3_d ))
        
        plt.fill_between(x_interval,y_up(x_interval),y_down(x_interval))
    
    
    
    # Separate the x and y coordinates (for the scatter function)
    x,y=zip(*coordinates)
    plt.scatter(x,y)
    # Plot the lattice vectors
    plt.arrow(0,0,v1[0,0],v1[1,0],color="red")
    plt.arrow(0,0,v2[0,0],v2[1,0],color="red")
    plt.xlim([-5, 10])
    plt.ylim([-5, 10])



    return plt.gcf

In [4]:
def interactive_lattice_plot(a1,a2,theta):
    plt.clf()
    '''
    :param a1,a2: norm of the lattice vectors
    :param theta: angle between the lattice vectors
    '''
    theta = theta/360*2*np.pi
    # prepare the vectors
    v2 = np.matrix([[0.00],[1.00]])
    v1 = np.matmul(np.matrix([[np.cos(-theta),-np.sin(-theta)],[np.sin(-theta),np.cos(-theta)]]),v2)
    
    v1*=a1
    v2*=a2
    
    plt.subplot(1,2,1)
    plt.axis([-6,6,-6,6])
    plt.axis('scaled')
    plt.axis('off')


    plot_lattice (v1,v2,24,24)

    plt.subplot(1,2,2)
    plt.axis([-6,6,-6,6])

    plt.axis('scaled')
    plt.axis('off')
    b1,b2=get_reciprocal_vectors(v1,v2)
    plot_lattice (b1*2/np.pi,b2*2/np.pi,24,24)


In [ ]:
interact(interactive_lattice_plot, a1=widgets.FloatSlider(min=1.,max=4.,step=0.1, value=1, description = 'a1'), a2=widgets.FloatSlider(min=1.,max=4.,step=0.1, value=1, description = 'a2'),theta=widgets.FloatSlider(min=30,max=120,step=1, value=90, description = 'theta'))


## Self-Study Questions

1. **Lattice Vectors:**
   - What are lattice vectors and how do they define a crystal lattice?
   - How would you describe a lattice in two and three dimensions?
   - What is the significance of the angle between the two lattice vectors in 2D?

2. **Reciprocal Lattice:**
   - Define the reciprocal lattice and explain its significance in crystallography.
   - Derive the expressions for the reciprocal lattice vectors in two dimensions.
   - How does the reciprocal lattice relate to the real space lattice in terms of symmetry and periodicity?

3. **Brillouin Zone:**
   - Define the first Brillouin Zone and explain its significance.
   - How is the Brillouin Zone related to the reciprocal lattice?
   - What are some physical phenomena that can be understood by studying the Brillouin Zone?

4. **Python Scripting and Visualization:**
   - Explain the purpose of the `get_reciprocal_vectors` function in the provided script. What mathematical operations are being performed to compute the reciprocal vectors?
   - Describe the steps involved in plotting the real space lattice using the `plot_lattice` function. How is the first Brillouin Zone plotted?
   - Explain how the interactive function `interactive_lattice_plot` works. What part of the code is responsible for updating the plot based on user interaction?
   - Modify the script to include labels for the real and reciprocal lattice vectors in the plot. How would you further improve the visualization for better understanding?

5. **Advanced Topics:**
   - Explore how the concepts of lattice, reciprocal lattice, and Brillouin Zone extend to three-dimensional crystalline materials.
   - Research on the concept of crystal planes and Miller indices. How are they related to the reciprocal lattice?
   - Investigate the relation between the Brillouin Zone and band structure in crystalline materials.
